## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"
[image2]: ./test_images/test1.jpg "Road Transformed"
[image3]: ./examples/binary_combo_example.jpg "Binary Example"
[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[image7]: ./output_images/test1.jpg "Undistored Image"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it!

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the first code cell of the IPython notebook located in "./examples/example.ipynb" (or in lines # through # of the file called `some_file.py`).  

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![alt text][image1]

I've used `cv2.getPerspectiveTransform()` to get M, the transform matrix. Afterwards, I finally used `cv2.warpPerspective()` to warp your image to a top-down view, whcih was not strictly required at this moment.

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib qt
%matplotlib inline

In [ ]:
# the number of inside corners in x & y
nx = 9
ny = 6

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(7,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    
    # If found, add object points, image points (after refining the them)
    if ret == True:
        objpoints.append(objp)
        corners_refined_loc = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners_refined_loc)
        
        # Calculate camera matrix and distortion coefficients
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        # Obtain undistored & warped image and matrix
        # 1) Undistort using mtx and dist
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        # 2) Convert to grayscale
        gray2 = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
        # 3) Find the chessboard corners
        ret2, corners2 = cv2.findChessboardCorners(gray2, (nx, ny), None)
        # 4) If corners found: 
        if ret2 == True:
            # a) draw corners
            cv2.drawChessboardCorners(undist, (nx, ny), corners2, ret2)
            # b) define 4 source points src = np.float32([[,],[,],[,],[,]])
            #Note: you could pick any four of the detected corners 
            # as long as those four corners define a rectangle
            #One especially smart way to do this would be to use four well-chosen
            # corners that were automatically detected during the undistortion steps
            #We recommend using the automatic detection of corners in your code
            offset = 100
            img_size = (gray2.shape[1], gray2.shape[0])
            src = np.float32([corners2[0], corners2[nx-1], corners2[-1], corners2[-nx]])
            # c) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
            dst = np.float32([[offset, offset], [img_size[0]-offset, offset], [img_size[0]-offset, img_size[1]-offset], [offset, img_size[1]-offset]])
            # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
            M = cv2.getPerspectiveTransform(src, dst)
            # e) use cv2.warpPerspective() to warp your image to a top-down view
            warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
        
            # Draw images
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
            f.tight_layout()
            ax1.imshow(img)
            ax1.set_title('Original Image: '+fname.split('/')[-1], fontsize=50)
            ax2.imshow(warped)
            ax2.set_title('Undistorted and Warped Image', fontsize=50)
            plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
            
            # Save undistored & warped images to ./output_images
            fname2 = './output_images/' + fname.split('/')[-1]
            cv2.imwrite(fname2, warped)
            #plt.imsave(fname2, warped)
        else:
            print(fname, ": corners NOT found in the undistored image")
    else:
        print(fname, ": corners NOT found in the original image.")

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:

[//]: # (Image References)

[image7]: ./output_images/test1.jpg "Undistored Image"

![alt text][image7]

In [ ]:
''''
Utilize pre-calculated camera matrix and distortion coefficients
to obtain distortion-corrected images from test images at './test_images/test*.jpg'
'''
# Make a list of calibration images
images = glob.glob('./test_images/test*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    
    # Obtain undistored & warped image and matrix
    # Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Draw images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image: '+fname.split('/')[-1], fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    # Save undistored & warped images to ./output_images
    fname2 = './output_images/' + 'undistorted_' + fname.split('/')[-1]
    cv2.imwrite(fname2, undist)
    #plt.imsave(fname2, undist)

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

pipeline()  function performs 6 jobs:
1. Convert to HLS color space instead of grayscale which lost color information for the lane lines.
2. Apply Sobel x to take derivative in x
3. Threshold x gradient
4. Threshold color channel
5. Stack each channel to view their individual contributions in green and blue respectively.
6. Combine the two binary thresholds

[//]: # (Image References)

[image3]: ./output_images/color_binary_test1.jpg "Color Binary Example"
[image4]: ./output_images/combined_binary_test1.jpg "Combined Binary Example"

![alt text][image3]
![alt text][image4]

In [ ]:
def binary_img(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    '''
    img is the undistorted image
    '''
    # 1. Convert to HLS color space and separate the L & S channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # 2. Sobel x
    # Take the derivative in x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)
    # Absolute x derivative to accentuate lines away from horizontal
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # 3. Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= s_thresh[1])] = 1
    
    # 4. Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # 5. Stack each channel to view their individual contribution in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors.
    color_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    # 6. Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary ==1) | (sxbinary == 1)] = 1
    
    return color_binary, combined_binary

'''
Apply pipeline() to undistored images of ./output_images/test*.jpg.
'''
# Make a list of calibration images
images = glob.glob('./output_images/undistorted_test*.jpg')

# Step through the list of undistorted images
for fname in images:
    # Read in undistorted image
    img = cv2.imread(fname)
    
    # Obtain color_binary & combined_binary
    color_binary, combined_binary = binary_img(img)
    # Draw images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(color_binary)
    ax1.set_title('Color binary Image: '+fname.split('/')[-1], fontsize=50)
    ax2.imshow(combined_binary)
    ax2.set_title('Combined binary Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    # Save color_binary & combined_binary images to ./output_images
    fname2 = './output_images/' + 'color_binary_' + fname.split('/')[-1]
    #cv2.imwrite(fname2, color_binary)
    plt.imsave(fname2, color_binary)
    
    fname3 = './output_images/' + 'combined_binary_' + fname.split('/')[-1]
    #cv2.imwrite(fname3, combined_binary)
    plt.imsave(fname3, combined_binary)

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `warper()`.  The `warper()` function takes as inputs an image (`img`) and calculates source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 585, 460      | 320, 0        | 
| 203, 720      | 320, 720      |
| 1127, 720     | 960, 720      |
| 695, 460      | 960, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

[image]: ./output_images/warped_test1.jpg "Wapred Image"

![alt text][image]

In [ ]:
def warper(img):
    '''
    img is undistroted image
    '''
    img_size = (img.shape[1], img.shape[0])
        
    # Define 4 source points src = np.float32([[,],[,],[,],[,]])
    src = np.float32([[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
                                  [((img_size[0] / 6) - 10), img_size[1]],
                                  [(img_size[0] * 5 / 6) + 60, img_size[1]],
                                  [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
    # Define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    dst = np.float32([[(img_size[0] / 4), 0],
                                  [(img_size[0] / 4), img_size[1]],
                                  [(img_size[0] * 3 / 4), img_size[1]],
                                  [(img_size[0] * 3 / 4), 0]])
    # Use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Use cv2.getPerspectiveTransform() to get Minv, the inverse transform matrix
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv

In [ ]:
''''
Utilize warper() fucntion
to obtain perspective-transformed, warped images from test images at './test_images/test*.jpg'
'''
# Make a list of calibration images
#images = glob.glob('./test_images/test*.jpg')
images = glob.glob('./output_images/undistorted_test*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    
    # Call warper() to obtained perspective-transformed, warped image
    warped, M, Minv = warper(img)
    
    # Draw images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image: '+fname.split('/')[-1], fontsize=50)
    ax2.imshow(warped)
    ax2.set_title('Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    # Save warped images to ./output_images
    fname2 = './output_images/' + 'warped_' + fname.split('/')[-1]
    cv2.imwrite(fname2, warped)
    #plt.imsave(fname2, undist)

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

I used sliding window method in `find_lane_pixels()` to find lane pixels from warped combined binary images. Afterwards, I fit my lane lines with a 2nd order polynomial using `np.polyfit` in `fit_polynomial()`. Once I've checked the lane with sliding window technique, with the known polynomial fit values, I implemented `fit_poly()` function assuming line location does not deviated far off frame by frame.

[//]: # (Image References)

[image4]: ./output_images/combined_bin_lane_polyfit_test6.jpg "Combined binary polynomial fit"

![alt text][image4]

In [ ]:
'''
Hyperparameters
'''
# Choose the number of sliding windows
nwindows = 9
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter windows
minpix = 50

# Polynomial fit values from the previous frame
# Make sure to grab the actual values from the previous step in your project!
#left_fit = np.array([ 2.13935315e-04, -3.77507980e-01,  4.76902175e+02])
#right_fit = np.array([4.17622148e-04, -4.93848953e-01,  1.11806170e+03])
left_fit = np.array([0, 0, 0])
right_fit = np.array([0, 0, 0])

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    #print("binary_warped.shape: ", binary_warped.shape)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    #print("histogram.shape: ", histogram.shape)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Find the four below boundaries of the window
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low), (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        #print("good_right_inds: ", good_right_inds)
        #print("good_right_inds.shape: ", good_right_inds.shape)
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If > minpix pixels, recenter next window
        # ('right' or 'leftx_current') on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [ ]:
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using 'np.polyfit'
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit, out_img

In [ ]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Calc both polynomials using ploty, left_fit and right_fit
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

In [ ]:
def search_around_poly(binary_warped):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, ploty

In [ ]:
''''
Utilize find_lane_pixels() and fit_polynomial() functions
to find and fit polynomials on the lane
'''
# Make a list of calibration images
images = glob.glob('./test_images/test*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    
    # Obtain undistored & warped image and matrix
    # Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Obtain color_binary & combined_binary
    color_binary, combined_binary = binary_img(undist)
    # Warpe binary images
    #warped_color_binary = warper(color_binary)
    warped_combined_binary, M, Minv = warper(combined_binary)
    # Fit polynomials on lanes
    #lane_poly_color = fit_polynomial(warped_color_binary)
    left_fit, right_fit, lane_poly_combined = fit_polynomial(warped_combined_binary)
    lane_poly_combined, left_fitx, right_fitx, ploty = search_around_poly(warped_combined_binary)
    
    # Draw images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(warped_combined_binary)
    ax1.set_title('Warped combined binary, ' + fname.split('/')[-1], fontsize=50)
    ax2.imshow(lane_poly_combined)
    ax2.set_title('Combined binary lane poly', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    # Save images to ./output_images
    #fname2 = './output_images/' + 'color_bin_lane_polyfit_' + fname.split('/')[-1]
    #cv2.imwrite(fname2, lane_poly_color)
    #plt.imsave(fname2, lane_poly_color)
    fname3 = './output_images/' + 'combined_bin_lane_polyfit_' + fname.split('/')[-1]
    #cv2.imwrite(fname3, lane_poly_combined)
    plt.imsave(fname3, lane_poly_combined)

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I took the meter per pixel ratio and implementation  from the lecture.

In [ ]:
'''
Hyperparameters
'''
# Define conversions in x and y from pixels space to meters
# meters per pixel in y dimension
ym_per_pix = 30/720
# meters per pixel in x dimension
xm_per_pix = 3.7/700

In [ ]:
def measure_curvature_real(ploty, left_fitx, right_fitx):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)*ym_per_pix
    
    # Fit a second order polynomial to pixel positions in each lane line
    # Fit new polynomials to x, y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1+ (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + 2*(right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Assume the car is at the center of the image
    dist_center = ((right_fitx[0]-left_fitx[0])/2 + left_fitx[0]) - img.shape[1]/2
    dist_center *= xm_per_pix
    
    return left_curverad, right_curverad, dist_center

In [ ]:
# Calculate the radius of curvature in meters for both lane lines
left_curverad, right_curverad, dist_center = measure_curvature_real(ploty, left_fitx, right_fitx)
print(left_curverad, 'm', right_curverad, 'm')
print("The distance from center fo lane: ", dist_center, 'm')

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

Here is an example of my result on a test image:

[//]: # (Image References)

[image6]: ./output_images/lane_clarified_test1.jpg "Lane area clarified"

![alt text][image6]

In [ ]:
def color_lane_area(undist, binary_warped, Minv):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    warped = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(warped)
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_pts = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    # Combine points from left and right
    pts = np.hstack((left_line_pts, right_line_pts))
    # Draw the lane onto the warped blank image with differently colored edge lines
    cv2.fillPoly(window_img, np.int_([pts]), (0,255, 0))
    cv2.polylines(window_img, np.int32([left_line_pts]), isClosed=False, color=(255,0,0), thickness=15)
    cv2.polylines(window_img, np.int32([right_line_pts]), isClosed=False, color=(0,0,255), thickness=15)
    # Combine area with differently colored edge lines
    combined = cv2.addWeighted(warped, 1, window_img, 0.5, 0)
    # Warp combined image to original image space using inverse perspective matrix (Minv)
    warped = cv2.warpPerspective(combined, Minv, (binary_warped.shape[1], binary_warped.shape[0]), flags=cv2.INTER_LINEAR)
    # Combine warped image onto undistorted image
    result = cv2.addWeighted(undist, 1, warped, 0.5, 0)
    
    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad, dist_center = measure_curvature_real(ploty, left_fitx, right_fitx)
    # Put texts on the image
    text = 'Left curve radius: ' + '{:04.2f}'.format(left_curverad) + 'm'
    cv2.putText(result, text, (40,70), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2, cv2.LINE_AA)
    text = 'Right curve radius: ' + '{:04.2f}'.format(right_curverad) + 'm'
    cv2.putText(result, text, (40,110), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2, cv2.LINE_AA)
    text = 'Distance from center: ' + '{:04.2f}'.format(dist_center) + 'm'
    cv2.putText(result, text, (40,150), cv2.FONT_HERSHEY_PLAIN, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, ploty

In [ ]:
''''
Utilize color_lane_area() functions
to find and fit polynomials on the lane
'''
# Make a list of calibration images
images = glob.glob('./test_images/test*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    
    # Obtain undistored & warped image and matrix
    # Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Obtain color_binary & combined_binary
    color_binary, combined_binary = binary_img(undist)
    # Warpe binary images
    #warped_color_binary = warper(color_binary)
    warped_combined_binary, M, Minv = warper(combined_binary)
    # Fit polynomials on lanes and color the lane area
    lane_clarified, left_fitx, right_fitx, ploty = color_lane_area(undist, warped_combined_binary, Minv)
    
    # Draw images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistored image , ' + fname.split('/')[-1], fontsize=50)
    ax2.imshow(lane_clarified)
    ax2.set_title('Lane area clarified', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    # Save images to ./output_images
    #fname2 = './output_images/' + 'color_bin_lane_polyfit_' + fname.split('/')[-1]
    #cv2.imwrite(fname2, lane_poly_color)
    #plt.imsave(fname2, lane_poly_color)
    fname3 = './output_images/' + 'lane_clarified_' + fname.split('/')[-1]
    #cv2.imwrite(fname3, lane_poly_combined)
    plt.imsave(fname3, lane_clarified)

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./output_images/project_video.mp4)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def pipeline(img):
    # Obtain undistored & warped image and matrix
    # Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Obtain color_binary & combined_binary
    color_binary, combined_binary = binary_img(undist)
    # Warpe binary images
    #warped_color_binary = warper(color_binary)
    warped_combined_binary, M, Minv = warper(combined_binary)
    # Fit polynomials on lanes and color the lane area
    lane_clarified, left_fitx, right_fitx, ploty = color_lane_area(undist, warped_combined_binary, Minv)
    
    return lane_clarified

In [ ]:
project_output = './output_images/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(project_output, audio=False)

In [ ]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [ ]:
challenge_output = './output_images/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [ ]:
harder_challenge_output = './output_images/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(harder_challenge_output, audio=False)

In [ ]:
HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  

1. Getting 4 points from `src` and `dst` correct in perspective transformation was the biggest challenge
     > I took what was there as my solution. However, could not figure out how to do it myself.
2. Getting polynomial correct mapped on lane lines.
     > More curved harder to figure the correct polynomial